In [48]:
import nbconvert

In [17]:
%%file ../jupyter_nbconvert_config.py
import nbconvert

from pathlib import Path
class Custom(nbconvert.exporters.MarkdownExporter):
    def from_notebook_node(self, nb, resources=None, **kw):
        
        nb, resources = super().from_notebook_node(nb, resources, **kw)
        md = dict(resources['metadata'])
        path = (Path(md['path']) / md['name']).with_suffix('.ipynb')
        print(path, md, dict(resources))
        
        return "adgasdf", resources
    
try:
    c.NbConvertApp.export_format = f"jupyter_nbconvert_config.Custom"
except: ...

Overwriting ../jupyter_nbconvert_config.py


In [18]:
!pushd .. && jupyter nbconvert _notebooks/Untitled.ipynb

_notebooks\Untitled.ipynb {'name': 'Untitled', 'path': '_notebooks', 'modified_date': 'September 13, 2018'} {'config_dir': 'C:\\Users\\deathbeds\\.jupyter', 'unique_key': 'Untitled', 'output_files_dir': 'Untitled_files', 'metadata': defaultdict(None, {'name': 'Untitled', 'path': '_notebooks', 'modified_date': 'September 13, 2018'}), 'output_extension': '.md', 'outputs': {}, 'raw_mimetypes': ['text/markdown', 'text/html', ''], 'global_content_filter': {'include_code': True, 'include_markdown': True, 'include_raw': True, 'include_unknown': True, 'include_input': True, 'include_output': True, 'include_input_prompt': True, 'include_output_prompt': True, 'no_prompt': False}}


[NbConvertApp] Converting notebook _notebooks/Untitled.ipynb to jupyter_nbconvert_config.Custom
[NbConvertApp] Writing 7 bytes to _notebooks\Untitled.md


In [10]:
import git

In [19]:
repo = git.Repo('..')

In [21]:
repo.tree()

KeyError: "Blob or Tree named '_notebooks' not found"

In [39]:
t = repo.tree()

In [42]:
b = t.blobs[0]

In [44]:
b.abspath,

('C:\\Users\\deathbeds\\Quansight.github.io\\.gitignore',)